In [64]:
using Plots
using Statistics
using LinearAlgebra
using JSON

In [2]:
# ; wget http://ee263.stanford.edu/hw/color_perception_data.json

In [4]:
# ; wget http://ee263.stanford.edu/hw/one_bad_sensor.json

## 3.250 

*Human color perception is based on the responses of three different types of color light receptors, called cones. The three types of cones have different spectral-response characteristics, and are called L, M, and, S because they respond mainly to long, medium, and short wavelengths, respectively. In this problem we will divide the visible spectrum into 20 bands, and model the cones’ responses as follows:*

$L_{\text {cone }}=\sum_{i=1}^{20} l_{i} p_{i}, \quad M_{\text {cone }}=\sum_{i=1}^{20} m_{i} p_{i}, \quad S_{\text {cone }}=\sum_{i=1}^{20} s_{i} p_{i}$

*where $p_{i}$ is the incident power in the $i$ th wavelength band, and $l_{i}, m_{i}$ and $s_{i}$ are nonnegative constants that describe the spectral responses of the different cones. The perceived color is a complex function of the three cone responses, i.e., the vector $\left(L_{\text {cone }}, M_{\text {cone }}, S_{\text {cone }}\right),$ with different cone response vectors perceived as different colors. (Actual color perception is a bit more complicated than this, but the basic idea is right.)*

### a)  Metamers
*When are two light spectra, $p$ and $\tilde{p},$ visually indistinguishable? (Visually identical lights with different spectral power compositions are called metamers.)*

When the vector produced (L, M, S) is the same. When the A matrix A = (l m s)^T is not 1-1, has a null space larger than {0}

### b) Visual color matching
*In a color matching problem, an observer is shown a test light, and is asked to change the intensities of three primary lights until the sum of the primary lights looks like the test light. In other words, the observer is asked the find a spectrum of the form $p_{\text {match }}=a_{1} u+a_{2} v+a_{3} w$ where $u, v, w$ are the spectra of the primary lights, and $a_{i}$ are the intensities to be found, that is visually indistinguishable from a given test light spectrum $p_{\text {test. }}$ Can this always be done? Discuss briefly.*

Only if the range R(A) lies in R3, the output space. This is a control problem, adjusting inputs to match a specific output. 

### c) Visual matching with phosphors
*A computer monitor has three phosphors, R, G, and B.  It is desired to adjust the phosphor intensities to create a color that looks like a reference test light.  Find weights that achieve the match or explain why no such weights exist. The data for this problem is in color_perception_data.json, which contains the vectors wavelength,B_phosphor,G_phosphor,R_phosphor,L_coefficients,M_coefficients,S_coefficients, and test_light.*

In [92]:
include("readclassjson.jl")

colors = readclassjson("data/color_perception_data.json")

Dict{Any,Any} with 10 entries:
  "test_light"     => [58.2792, 42.3496, 51.5512, 33.3951, 43.2907, 22.595, 57.…
  "wavelength"     => [380.0, 400.0, 420.0, 440.0, 460.0, 480.0, 500.0, 520.0, …
  "tungsten"       => [20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 1…
  "L_coefficients" => [0.12589, 0.079433, 0.1, 0.15849, 0.19953, 0.25119, 0.316…
  "M_coefficients" => [0.15849, 0.14125, 0.14125, 0.25119, 0.31623, 0.39811, 0.…
  "G_phosphor"     => [21.0, 23.0, 26.0, 30.0, 35.0, 45.0, 75.0, 90.0, 100.0, 9…
  "R_phosphor"     => [15.0, 16.0, 17.0, 18.0, 19.0, 21.0, 23.0, 26.0, 30.0, 35…
  "B_phosphor"     => [30.0, 35.0, 45.0, 75.0, 90.0, 100.0, 90.0, 75.0, 45.0, 3…
  "S_coefficients" => [0.39811, 0.50119, 0.63096, 0.79433, 0.50119, 0.25119, 0.…
  "sunlight"       => [40.0, 70.0, 100.0, 160.0, 240.0, 220.0, 180.0, 160.0, 18…

In [49]:
test = colors["test_light"]
w = colors["wavelength"]
rgb = [colors["R_phosphor"] colors["G_phosphor"] colors["B_phosphor"]]
lms = [colors["L_coefficients"] colors["M_coefficients"] colors["S_coefficients"]];

In [40]:
y = transpose(lms) * test

3-element Array{Float64,1}:
 191.16611560329102
 242.7872917703638
 135.92624672925962

In [41]:
A = transpose(lms) * rgb

3×3 Array{Float64,2}:
 145.555   254.398   197.777
 148.341   333.869   278.369
  55.7972   95.5058  194.713

In [47]:
invA = inv(A)
x = invA * y

3-element Array{Float64,1}:
 0.4225929938835502
 0.09874256378706159
 0.5285525473174244

In [52]:
A*x

3-element Array{Float64,1}:
 191.16611560329105
 242.78729177036385
 135.92624672925962

In [51]:
isapprox(A*x,y)

true

### d)
Effects of illumination. An object's surface can be characterized by its reflectance ( $i . e .,$ the fraction of light it reflects) for each band of wavelengths. If the object is illuminated with a light spectrum characterized by $I_{i},$ and the reflectance of the object is $r_{i}($ which is between 0 and 1 ), then the reflected light spectrum is given by $I_{i} r_{i},$ where $i=1, \ldots, 20$ denotes the wavelength band. Now consider two objects illuminated (at different times) by two different light sources, say an incandescent bulb and sunlight. Sally argues that if the two objects look identical when illuminated by a tungsten bulb, then they will look identical when illuminated by sunlight. Beth disagrees: she says that two objects can appear identical when illuminated by a tungsten bulb, but look different when lit by sunlight. Who is right? If Sally is right, explain why. If Beth is right give an example of two objects that appear identical under one light source and different under another. You can use the vectors sunlight and tungsten defined in the data file as the light
sources.

In [54]:
tungsten = colors["tungsten"]
sunlight = colors["sunlight"];

In [55]:
transpose(lms) * tungsten

3-element Array{Float64,1}:
 370.283327
 425.851782523
 142.47821435667242

In [223]:
B = transpose(lms);

In [229]:
r1 = [repeat([0.5], 10) ; repeat([0.1], 10)]
IR1 = (tungsten .* r1);

In [241]:
nullVector = nullspace(B)[:,1];

In [238]:
r2 = (IR1 - 50 .* nullVector) ./ tungsten
IR2 = (tungsten .* r2);

In [246]:
isapprox(r1, r2)

false

In [242]:
tungstenLook1 = B * IR1

3-element Array{Float64,1}:
 124.82236869999998
 173.2441382523
  71.23471167566724

In [243]:
tungstenLook2 = B * IR2

3-element Array{Float64,1}:
 124.82236869999997
 173.2441382523
  71.23471167566724

In [244]:
isapprox(tungstenLook1, tungstenLook2)

true

In [245]:
sun_IR1 = (sunlight .* r1);
sun_IR2 = (sunlight .* r2);

In [247]:
sunLook1 = B * sun_IR1

3-element Array{Float64,1}:
 247.85898313
 359.8846318979
 215.66132988271158

In [248]:
sunLook2 = B * sun_IR2

3-element Array{Float64,1}:
 246.60513901099375
 357.20225222205795
 196.89366873222457

In [249]:
isapprox(sunLook1, sunLook2)

false

## 3.430. Single sensor failure detection and identification
Single sensor failure detection and identification. We have $y=A x,$ where $A \in \mathbb{R}^{m \times n}$ is known, and $x \in \mathbb{R}^{n}$ is to be found. Unfortunately, up to one sensor may have failed (but you don't know which one has failed, or even whether any has failed). You are given $\tilde{y}$ and not $y,$ where $\tilde{y}$ is the same as $y$ in all entries except, possibly, one (say, the $k$ th entry). If all sensors are operating correctly, we have $y=\tilde{y} .$ If the $k$ th sensor fails, we have $\tilde{y}_{i}=y_{i}$ for all $i \neq k$
The file one_bad_sensor.m, available on the course web site, defines $A$ and $\tilde{y}$ (as A and ytilde). Determine which sensor has failed (or if no sensors have failed). You must explain your method, and submit your code.

For this exercise, you can use the matlab code rank ( $[\mathrm{F}$ g] ) = rank (F) to check if $g \in$ range $(F) .$ (We will see later a much better way to check if $g \in \operatorname{range}(F) .)$

In [145]:
sensor = readclassjson("data/one_bad_sensor.json")
A = sensor["A"]
ytilde = sensor["ytilde"];

In [261]:
size(A)

(15, 10)

In [262]:
@show rank(A)
@show rank(hcat(A,ytilde))

rank(A) = 10
rank(hcat(A, ytilde)) = 11


11

In [169]:
round5(x) = round(x, digits=5);

In [170]:
round5.(ytilde)

15-element Array{Float64,1}:
  0.2937
 -0.54803
  0.00353
  1.37586
 -6.75268
  1.19049
  8.7822
  1.91197
 -1.46287
 -4.43363
 -1.72341
 -4.54749
 -0.10925
  8.03353
  1.78262

In [287]:
hcat(A[1:10, :], ytilde[1:10])

10×11 Array{Float64,2}:
  1.16495    -0.360031    0.375042    …   0.438706   0.670293    0.293701
  0.62684    -0.135577    1.12516        -1.24734    0.420147   -0.548031
  0.0750806  -1.34934     0.728643        0.324668  -2.87275     0.00353264
  0.351608   -1.27045    -2.37745         0.390071   1.68587     1.37586
 -0.696514    0.984571   -0.273783       -0.405139   0.0279256  -6.75268
  1.69614    -0.0448816  -0.322941    …   0.292316  -0.902032    1.19049
  0.0590606  -0.798946    0.317989        2.56591   -2.05326     8.7822
  1.79707    -0.765173   -0.511173       -0.457817   0.0890866   1.91197
  0.26407     0.861736   -0.00204164     -1.61083    2.0871     -1.46287
  0.871674   -0.0562256   1.60651        -2.66952    0.365119   -4.43363

In [266]:
for i = 1:size(A)[1]
    @show rank(hcat(A[1:i, :],ytilde[1:i]))
#     @show size(A[1:end .!= i, :])
#     @show size(ytilde[1:end .!= i])
end

rank(hcat(A[1:i, :], ytilde[1:i])) = 1
rank(hcat(A[1:i, :], ytilde[1:i])) = 2
rank(hcat(A[1:i, :], ytilde[1:i])) = 3
rank(hcat(A[1:i, :], ytilde[1:i])) = 4
rank(hcat(A[1:i, :], ytilde[1:i])) = 5
rank(hcat(A[1:i, :], ytilde[1:i])) = 6
rank(hcat(A[1:i, :], ytilde[1:i])) = 7
rank(hcat(A[1:i, :], ytilde[1:i])) = 8
rank(hcat(A[1:i, :], ytilde[1:i])) = 9
rank(hcat(A[1:i, :], ytilde[1:i])) = 10
rank(hcat(A[1:i, :], ytilde[1:i])) = 11
rank(hcat(A[1:i, :], ytilde[1:i])) = 11
rank(hcat(A[1:i, :], ytilde[1:i])) = 11
rank(hcat(A[1:i, :], ytilde[1:i])) = 11
rank(hcat(A[1:i, :], ytilde[1:i])) = 11


In [311]:
1:15 .∈ [[1,2,3]]

15-element BitArray{1}:
 1
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [326]:
for i = 1:size(A)[1]
#     A_prime = A[1:end .!= i, :]
#     y_prime = ytilde[1:end .!= i]
    idx = 1:15 .∉ [[i-2,i-1, i, i+1]]
    A_prime = A[idx, :]
    y_prime = ytilde[idx]
 
#     @show 1:15 .!= i
#     @show rank(hcat(A_prime,y_prime))
#     @show size(y_prime)
    if rank(hcat(A_prime,y_prime)) == rank(A_prime)
        println(i)
#         println(rank(hcat(A_prime,y_prime)))
#         println(rank(A_prime))
    end


end


rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11
rank(hcat(A_prime, y_prime)) = 11


In [322]:
1:15 .∉ [[i-2, i-1, i, i+1]]

15-element BitArray{1}:
 0
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [323]:
A_prime = A[1:15 .∉ [[i-2,i-1, i, i+1]], :]
y_prime = ytilde[1:15 .∉ [[i-2, i-1, i, i+1]]]

13-element Array{Float64,1}:
  0.00353264
  1.37586064
 -6.75268364
  1.19048564
  8.78219664
  1.91197364
 -1.46286864
 -4.43362564
 -1.72340564
 -4.54749364
 -0.10924664
  8.03352764
  1.78262064